# Week 4 Exercise — Mini meal / calorie planner

A **tool-using agent** that helps plan meals and calories: compute TDEE and macro targets, suggest meals from an in-memory recipe list, and build a grocery list. Inspired by the [Fitness & nutrition planner](https://github.com/llm_engineering/community-contributions/tree/main/fitness-nutrition-planner-agent) community contribution.

**Tools:** `calc_tdee`, `suggest_meal`, `grocery_list`  
**UI:** Gradio chat. **API:** OpenRouter (or OpenAI) as in week 2/3.

In [1]:
# imports

import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr

In [2]:
# environment & API client (OpenRouter preferred, fallback OpenAI)

load_dotenv(override=True)
openrouter_api_key = os.getenv("OPENROUTER_API_KEY")
openai_api_key = os.getenv("OPENAI_API_KEY")

if openrouter_api_key and (openrouter_api_key.startswith("sk-or-") or openrouter_api_key.startswith("sk-proj-")):
    client = OpenAI(api_key=openrouter_api_key, base_url="https://openrouter.ai/api/v1")
    MODEL = "openai/gpt-4o-mini"
    print("Using OpenRouter.")
elif openai_api_key:
    client = OpenAI(api_key=openai_api_key)
    MODEL = "gpt-4o-mini"
    print("Using OpenAI.")
else:
    client = OpenAI()
    MODEL = "gpt-4o-mini"
    print("Using default client (set OPENROUTER_API_KEY or OPENAI_API_KEY in .env).")

Using OpenRouter.


## In-memory recipe list

Simple list of meals with name, kcal, ingredients, and tags (e.g. vegetarian, breakfast). The agent uses these for `suggest_meal` and `grocery_list`.

In [3]:
# In-memory recipe "DB" (extend or replace with a real API later)

RECIPES = [
    {"name": "Greek Yogurt Parfait", "kcal": 380, "tags": ["vegetarian", "breakfast"],
     "ingredients": [{"item": "greek yogurt", "qty": 200, "unit": "g"}, {"item": "berries", "qty": 150, "unit": "g"}, {"item": "granola", "qty": 30, "unit": "g"}]},
    {"name": "Tofu Veggie Stir-Fry with Rice", "kcal": 650, "tags": ["vegan", "dinner"],
     "ingredients": [{"item": "firm tofu", "qty": 150, "unit": "g"}, {"item": "mixed vegetables", "qty": 200, "unit": "g"}, {"item": "brown rice (cooked)", "qty": 200, "unit": "g"}]},
    {"name": "Lentil Soup with Bread", "kcal": 520, "tags": ["vegan", "lunch"],
     "ingredients": [{"item": "lentils (cooked)", "qty": 200, "unit": "g"}, {"item": "vegetable broth", "qty": 400, "unit": "ml"}, {"item": "wholegrain bread", "qty": 60, "unit": "g"}]},
    {"name": "Chicken Quinoa Bowl", "kcal": 620, "tags": ["gluten-free", "dinner"],
     "ingredients": [{"item": "chicken breast", "qty": 140, "unit": "g"}, {"item": "quinoa (cooked)", "qty": 185, "unit": "g"}, {"item": "spinach", "qty": 60, "unit": "g"}]},
    {"name": "Salmon, Potatoes & Greens", "kcal": 680, "tags": ["gluten-free", "dinner", "fish"],
     "ingredients": [{"item": "salmon fillet", "qty": 150, "unit": "g"}, {"item": "potatoes", "qty": 200, "unit": "g"}, {"item": "broccoli", "qty": 150, "unit": "g"}]},
    {"name": "Cottage Cheese Bowl", "kcal": 380, "tags": ["vegetarian", "breakfast", "high-protein"],
     "ingredients": [{"item": "cottage cheese", "qty": 200, "unit": "g"}, {"item": "pineapple", "qty": 150, "unit": "g"}]},
    {"name": "Oatmeal with Banana", "kcal": 350, "tags": ["vegan", "vegetarian", "breakfast"],
     "ingredients": [{"item": "rolled oats", "qty": 80, "unit": "g"}, {"item": "banana", "qty": 1, "unit": "unit"}, {"item": "almond milk", "qty": 120, "unit": "ml"}]},
    {"name": "Hummus & Veggie Wrap", "kcal": 420, "tags": ["vegan", "vegetarian", "lunch"],
     "ingredients": [{"item": "whole wheat wrap", "qty": 1, "unit": "unit"}, {"item": "hummus", "qty": 80, "unit": "g"}, {"item": "cucumber", "qty": 50, "unit": "g"}, {"item": "tomato", "qty": 50, "unit": "g"}]},
]

## Tool 1: Calculate TDEE and macro targets

Uses Mifflin–St Jeor equation + activity factor. Returns target kcal and suggested protein/carbs/fat split.

In [4]:
ACTIVITY_FACTORS = {"sedentary": 1.2, "light": 1.375, "moderate": 1.55, "active": 1.725, "very_active": 1.9}

def calc_tdee(age: int, weight_kg: float, height_cm: float, activity: str, sex: str = "female"):
    """Compute TDEE (Total Daily Energy Expenditure) and macro targets. Activity: sedentary, light, moderate, active, very_active."""
    # BMR (Mifflin–St Jeor)
    if sex.lower().startswith("m"):
        bmr = 10 * weight_kg + 6.25 * height_cm - 5 * age + 5
    else:
        bmr = 10 * weight_kg + 6.25 * height_cm - 5 * age - 161
    factor = ACTIVITY_FACTORS.get(activity.lower(), 1.2)
    tdee = int(round(bmr * factor))
    target_kcal = tdee
    protein_g = int(round(target_kcal * 0.30 / 4))
    carbs_g = int(round(target_kcal * 0.40 / 4))
    fat_g = int(round(target_kcal * 0.30 / 9))
    return json.dumps({"tdee": tdee, "target_kcal": target_kcal, "protein_g": protein_g, "carbs_g": carbs_g, "fat_g": fat_g})

## Tool 2: Suggest meals

Filter recipes by dietary prefs (e.g. vegetarian, vegan) and target calories per meal; return matching meals.

In [5]:
def suggest_meal(calories_target: int, dietary_prefs: str = ""):
    """Suggest meals from the recipe list that fit the calorie target and dietary preferences (e.g. vegetarian, vegan)."""
    prefs = (dietary_prefs or "").lower().split()
    candidates = []
    for r in RECIPES:
        tags_lower = [t.lower() for t in r["tags"]]
        if prefs and not any(p in tags_lower for p in prefs):
            continue
        candidates.append({"name": r["name"], "kcal": r["kcal"], "tags": r["tags"]})
    if not candidates:
        candidates = [{"name": r["name"], "kcal": r["kcal"], "tags": r["tags"]} for r in RECIPES]
    # Sort by closeness to target and return top options
    candidates.sort(key=lambda x: abs(x["kcal"] - calories_target))
    return json.dumps(candidates[:8])

## Tool 3: Grocery list from meal names

Aggregate ingredients for the given meal names and return a consolidated list.

In [6]:
def grocery_list(meal_names: list):
    """Build a consolidated grocery list from a list of meal names (must match RECIPES names)."""
    name_to_recipe = {r["name"]: r for r in RECIPES}
    agg = {}
    for name in meal_names:
        r = name_to_recipe.get(name)
        if not r:
            continue
        for ing in r.get("ingredients", []):
            key = (ing["item"].lower(), ing.get("unit", ""))
            agg[key] = agg.get(key, 0) + float(ing.get("qty", 0))
    lines = []
    for (item, unit), qty in sorted(agg.items()):
        lines.append(f"- {item}: {qty} {unit}")
    return "\n".join(lines) if lines else "No ingredients found for those meals."

## Tool schemas (OpenAI function-calling format)

In [7]:
calc_tdee_function = {
    "name": "calc_tdee",
    "description": "Compute TDEE and macro targets (target_kcal, protein_g, carbs_g, fat_g) from age, weight_kg, height_cm, activity level, and sex.",
    "parameters": {
        "type": "object",
        "properties": {
            "age": {"type": "integer", "description": "Age in years"},
            "weight_kg": {"type": "number", "description": "Weight in kilograms"},
            "height_cm": {"type": "number", "description": "Height in centimeters"},
            "activity": {"type": "string", "description": "Activity level: sedentary, light, moderate, active, very_active"},
            "sex": {"type": "string", "description": "Sex for BMR: male or female", "default": "female"},
        },
        "required": ["age", "weight_kg", "height_cm", "activity"],
        "additionalProperties": False,
    },
}

suggest_meal_function = {
    "name": "suggest_meal",
    "description": "Suggest meals from the recipe database that fit a calorie target and optional dietary preferences (e.g. vegetarian, vegan).",
    "parameters": {
        "type": "object",
        "properties": {
            "calories_target": {"type": "integer", "description": "Target calories per meal"},
            "dietary_prefs": {"type": "string", "description": "Dietary preferences, e.g. vegetarian or vegan", "default": ""},
        },
        "required": ["calories_target"],
        "additionalProperties": False,
    },
}

grocery_list_function = {
    "name": "grocery_list",
    "description": "Build a consolidated grocery list from a list of meal names (exact names from suggest_meal).",
    "parameters": {
        "type": "object",
        "properties": {
            "meal_names": {"type": "array", "items": {"type": "string"}, "description": "List of meal names"},
        },
        "required": ["meal_names"],
        "additionalProperties": False,
    },
}

tools = [
    {"type": "function", "function": calc_tdee_function},
    {"type": "function", "function": suggest_meal_function},
    {"type": "function", "function": grocery_list_function},
]

## Tool dispatcher and agent loop

Convert Gradio history to messages, run the chat with tools, and loop until the model stops calling tools.

In [8]:
def handle_tool_calls(message):
    responses = []
    for tool_call in message.tool_calls:
        name = tool_call.function.name
        args = json.loads(tool_call.function.arguments or "{}")
        if name == "calc_tdee":
            result = calc_tdee(
                age=args.get("age"),
                weight_kg=args.get("weight_kg"),
                height_cm=args.get("height_cm"),
                activity=args.get("activity", "moderate"),
                sex=args.get("sex", "female"),
            )
        elif name == "suggest_meal":
            result = suggest_meal(
                calories_target=args.get("calories_target"),
                dietary_prefs=args.get("dietary_prefs", ""),
            )
        elif name == "grocery_list":
            result = grocery_list(meal_names=args.get("meal_names", []))
        else:
            result = json.dumps({"error": f"Unknown tool: {name}"})
        responses.append({"role": "tool", "content": result, "tool_call_id": tool_call.id})
    return responses

In [9]:
SYSTEM_PROMPT = """You are a helpful meal and calorie planning assistant. You have access to these tools:
- calc_tdee: when the user gives age, weight, height, and activity level, use it to compute their TDEE and macro targets.
- suggest_meal: to suggest meals that fit a calorie target and dietary preferences (e.g. vegetarian, vegan).
- grocery_list: to build a consolidated grocery list from a list of meal names.
Use the tools when relevant. For example: if the user says they need ~2000 kcal/day and are vegetarian, first compute targets (or use 2000 as target), then suggest meals for breakfast/lunch/dinner (e.g. ~400–700 kcal per meal), then optionally build a grocery list from the chosen meals. Keep answers concise and structured."""

In [10]:
def _history_to_messages(history):
    # Gradio can pass history as list of [user, assistant] tuples OR list of {role, content} dicts.
    out = []
    if not history:
        return out
    for item in history:
        if isinstance(item, dict) and "role" in item:
            content = item.get("content") or ""
            if content:
                out.append({"role": item["role"], "content": content})
        else:
            try:
                user_msg = item[0] if len(item) > 0 else ""
                assistant_msg = item[1] if len(item) > 1 else ""
            except (TypeError, IndexError):
                continue
            if user_msg:
                out.append({"role": "user", "content": str(user_msg)})
            if assistant_msg:
                out.append({"role": "assistant", "content": str(assistant_msg)})
    return out

def chat(message, history):
    messages = [{"role": "system", "content": SYSTEM_PROMPT}] + _history_to_messages(history) + [{"role": "user", "content": message}]
    response = client.chat.completions.create(model=MODEL, messages=messages, tools=tools, tool_choice="auto")

    while response.choices[0].finish_reason == "tool_calls":
        msg = response.choices[0].message
        tool_responses = handle_tool_calls(msg)
        assistant_msg = {"role": "assistant", "content": msg.content or None}
        assistant_msg["tool_calls"] = [{"id": tc.id, "type": getattr(tc, "type", "function"), "function": {"name": tc.function.name, "arguments": tc.function.arguments}} for tc in msg.tool_calls]
        messages.append(assistant_msg)
        messages.extend(tool_responses)
        response = client.chat.completions.create(model=MODEL, messages=messages, tools=tools, tool_choice="auto")

    return response.choices[0].message.content or "(No response)"

## Launch Gradio UI

In [13]:
gr.ChatInterface(fn=chat, title="Mini meal / calorie planner").launch(inbrowser=True, theme=gr.themes.Soft())

* Running on local URL:  http://127.0.0.1:7887
* To create a public link, set `share=True` in `launch()`.
